In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model

In [2]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [3]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer_with_fedNova import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Projects/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Projects/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [5]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("federatedscope/gfl/baseline/laplacian_gine_on_cikmcup.yaml")
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
#cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']

cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance=1e4
cfg.params.eps=1e-15
cfg.params.p=0.

#### configure other options

In [6]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = \
    'Laplacian_fednova_batch_test'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 200

cfg.federate.client_num = 8
cfg.early_stop.patience = 20000
#cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 5.0
#cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [7]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(False)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [8]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline" \
                  "/laplacian_gine_on_cikmcup_per_client_with_alpha" \
                  ".yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [9]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [10]:
from federatedscope.contrib.workers.laplacian_fednova_client import LaplacianFedNovaClient
from federatedscope.contrib.workers.laplacian_fednova_server import LaplacianServer

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-10-21 17:03:36,481 (utils:129)INFO: the current machine is at 127.0.1.1
2022-10-21 17:03:36,482 (utils:131)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-10-21 17:03:36,483 (utils:132)INFO: the output dir is exp/Laplacian_fednova_batch_test_gin_on_cikmcup_lr0.1_lstep10_/sub_exp_20221021170336
2022-10-21 17:03:36,541 (cikm_cup:57)INFO: Loading CIKMCUP data from /home/michael/Projects/CKIM_Competition/data/CIKM22Competition.
2022-10-21 17:03:36,543 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #1.
2022-10-21 17:03:36,996 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #2.
2022-10-21 17:03:37,036 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #3.
2022-10-21 17:03:37,551 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #4.
2022-10-21 17:03:37,586 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #5.
2022-10-21 17:03:37,628 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #6.
2022-10-21 17:03:38,019 (cikm_cup:67)INFO: Loading CIKMCUP data for Cli

In [11]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianFedNovaClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

server params: 
encoder_atom.atom_embedding_list.0.weight


2022-10-21 17:03:40,755 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-10-21 17:03:40,772 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: 

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
gnn.convs.0.nn.linears.0.weight
gnn.convs.0.nn.linears.0.bias
gnn.convs.0.nn.l

2022-10-21 17:03:40,952 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_with_fedNova.LaplacianFedNovaTrainer'>
2022-10-21 17:03:40,953 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-10-21 17:03:40,970 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 8
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/

[0.6369975934709822, 0.6369975934709822, 7.431638590494791, 1.7149935208834135, 11.147457885742188, 1.1147457885742187, 1.2386064317491319, 7.431638590494791]


...

[1.114745864868164, 11.14745864868164, 1.7149936382587139, 7.431639099121094, 7.431639099121094, 1.2386065165201823, 0.6369976370675223, 0.6369976370675223]


2022-10-21 18:22:55,411 (client:410)INFO: {'Role': 'Client #1', 'Round': 44, 'Results_raw': {'test_total': 417, 'test_loss': 780.217768, 'test_avg_loss': 1.871026, 'test_imp_ratio': -49.090891, 'val_total': 416, 'val_loss': 68.032848, 'val_avg_loss': 0.16354, 'val_imp_ratio': 88.15341}}
2022-10-21 18:22:55,413 (monitor:512)INFO: current_best=88.15341, should_save=True
2022-10-21 18:22:55,414 (client:431)INFO: Client: #1, val_imp_ratio: 88.15341. model saved at exp/Laplacian_fednova_batch_test_gin_on_cikmcup_lr0.1_lstep10_/sub_exp_20221021170336/model1.pth
2022-10-21 18:22:56,405 (client:410)INFO: {'Role': 'Client #2', 'Round': 44, 'Results_raw': {'test_total': 61, 'test_loss': 34.02256, 'test_avg_loss': 0.557747, 'test_imp_ratio': 3.773423, 'val_total': 60, 'val_loss': 38.740886, 'val_avg_loss': 0.645681, 'val_imp_ratio': -9.339806}}
2022-10-21 18:22:56,407 (monitor:512)INFO: current_best=2.169647, should_save=False
2022-10-21 18:23:06,702 (client:410)INFO: {'Role': 'Client #3', 'Round

[0.6369975934709822, 1.1147457885742187, 11.147457885742188, 7.431638590494791, 0.6369975934709822, 1.7149935208834135, 7.431638590494791, 1.2386064317491319]


2022-10-21 18:24:40,110 (client:410)INFO: {'Role': 'Client #1', 'Round': 45, 'Results_raw': {'test_total': 417, 'test_loss': 795.256121, 'test_avg_loss': 1.907089, 'test_imp_ratio': -49.090891, 'val_total': 416, 'val_loss': 71.195428, 'val_avg_loss': 0.171143, 'val_imp_ratio': 85.419582}}
2022-10-21 18:24:40,111 (monitor:512)INFO: current_best=88.15341, should_save=False
2022-10-21 18:24:40,985 (client:410)INFO: {'Role': 'Client #2', 'Round': 45, 'Results_raw': {'test_total': 61, 'test_loss': 34.633432, 'test_avg_loss': 0.567761, 'test_imp_ratio': 3.773423, 'val_total': 60, 'val_loss': 38.96228, 'val_avg_loss': 0.649371, 'val_imp_ratio': -9.339806}}
2022-10-21 18:24:40,987 (monitor:512)INFO: current_best=2.169647, should_save=False
2022-10-21 18:24:55,160 (client:410)INFO: {'Role': 'Client #3', 'Round': 45, 'Results_raw': {'test_total': 740, 'test_loss': 801.347231, 'test_avg_loss': 1.082902, 'test_imp_ratio': -21.293255, 'val_total': 740, 'val_loss': 314.684465, 'val_avg_loss': 0.4252

KeyboardInterrupt: 

In [ ]:
data

In [ ]:
import os

In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']